### Василевская Вероника. Поток DAPR-180

## PROJECT 4: Исследование данных и подготовка рекомендательной системы для увеличения среднего чека для онлайн школы MasterMind.

### ЦЕЛЬ: 

 - Получить рекомендательную систему, благодаря которой можно будет предлагать клиентам интересные им курсы и тем самым повышать средний чек. 
 - Для изначального воплощения этой системы нужно составить таблицу, в которой курсам будет соответствовать по две рекомендации.


### Исходные данные: 

 ##### Таблица 'carts' — содержит данные о пользовательских корзинах:

Promo Code ID — ID промокода, если он есть  
Purchased At — дата оплаты  
User ID — ID пользователя  
Created At — дата создания корзины  
Updated At — дата последнего обновления информации  
ID — идентификатор корзины  
State — состояние оплаты  

##### Таблица 'cart items' — содержит данные о курсах, которые пользователи добавили в корзину

Created At — дата создания события  
Resource Type — тип продукта  
Resource ID — ID курса  
Cart ID — идентификатор корзины  
Updated At — дата последнего обновления информации  
ID — идентификатор операции

#### Для построения рекомендательной системы полезно знать, какие курсы чаще всего покупают вместе с конкретным курсом (необязательно в рамках одной корзины).

Для этого необходимо обработать данные, чтобы понять каковы были продажи курсов в разрезе пользователей.  
Для этого выясним:


1. Какое количество курсов есть у школы.

Для этого используем следующий SQL код:<br>

<blockquote><span style="color: blue;"> SELECT COUNT (DISTINCT resource_id) AS total_courses</span> <small><span style="color: grey;">-- количество уникальных id курсов добавленных в корзину</span></small></blockquote>  
<blockquote style="color: blue;">FROM final.cart_items AS course_id</blockquote>  
<blockquote><span style="color: blue;">JOIN final.carts ON items.cart_id = carts.id</span> <small><span style="color: grey;"> -- объединяем таблицы с данными по ключу 'id'</span></small></blockquote>  
<blockquote><span style="color: blue;">WHERE resource_type = 'Course';</span> <small><span style="color: grey;">--отфильтровываем только курсы по типу продукта</span></small></blockquote>  

Всего получается 127 уникальных курсов


2. Составим таблицу, содержащую данные сколько клиентов купили больше одного курса и какие именно курсы они купили.  

Для этого используем следующий SQL код:<br>

<blockquote style="color: grey;"><small>-- Шаг 1: Создание временной таблицы sum_courses</small></blockquote>  
<blockquote style="color: blue;">WITH sum_courses AS (</blockquote>  
    <blockquote style="margin-left: 20px; color: blue;">SELECT</blockquote>  
        <blockquote style="margin-left: 40px; color: blue;">user_id,</blockquote>  
        <blockquote style="margin-left: 40px; color: blue;">COUNT(resource_id) AS total_courses <small><span style="color: grey;"> -- количество уникальных id курсов добавленных в корзину</span></small></blockquote>  
    <blockquote style="margin-left: 20px; color: blue;">FROM</blockquote>
        <blockquote style="margin-left: 40px; color: blue;">final.cart_items AS items</blockquote> 
        <blockquote style="margin-left: 40px; color: blue;">JOIN final.carts ON items.cart_id = carts.id</span> <small><span style="color: grey;"> -- объединяем таблицы с исходными данными по ключу 'id'</span></small></blockquote> 
    <blockquote style="margin-left: 20px; color: blue;">WHERE</blockquote>
        <blockquote style="margin-left: 40px; color: blue;">resource_type = 'Course'</span> <small><span style="color: grey;">--отфильтровываем только курсы по типу продукта</span></small></blockquote>
        <blockquote style="margin-left: 40px; color: blue;">AND state = 'successful'</span> <small><span style="color: grey;">--отфильтровываем только оплаченные курсы</span></small></blockquote>
    <blockquote style="margin-left: 20px; color: blue;">GROUP BY 1</blockquote> 
    <blockquote style="margin-left: 20px; color: blue;">HAVING COUNT(DISTINCT resource_id) > 1</span> <small><span style="color: grey;">-- отфильтровываем пользователей, которые купили более чем один уникальный курс</span></small></blockquote> 
<blockquote style="color: blue;">)</blockquote>

<blockquote style="color: grey;"><small>-- Шаг 2: Получение таблицы c id пользователей, которые купили больше одного курса и списком id курсов, которые они купили</small></blockquote>
<blockquote style="color: blue;">SELECT</blockquote>
    <blockquote style="margin-left: 20px; color: blue;">sum_courses.user_id AS user_id,<small><span style="color: grey;"> -- id пользователей, купивших более одного курса</span></small></blockquote>
    <blockquote style="margin-left: 20px; color: blue;">items.resource_id AS resource_id<small><span style="color: grey;"> -- id курсов</span></small></blockquote>
<blockquote style="color: blue;">FROM</blockquote>
    <blockquote style="margin-left: 20px; color: blue;">sum_courses</blockquote>
    <blockquote style="margin-left: 20px; color: blue;">JOIN final.carts ON sum_courses.user_id = carts.user_id</span> <small><span style="color: grey;"> -- объединяем временную таблицу sum_courses с таблицей 'carts' по ключу 'id'</span></small></blockquote>
    <blockquote style="margin-left: 20px; color: blue;">JOIN final.cart_items AS items ON items.cart_id = carts.id</span> <small><span style="color: grey;"> -- объединяем таблицы с исходными данными по ключу 'id'</span></small></blockquote>
<blockquote style="color: blue;">WHERE</blockquote>
    <blockquote style="margin-left: 20px; color: blue;">state = 'successful'</span> <small><span style="color: grey;">--отфильтровываем только оплаченные курсы</span></small></blockquote>
    <blockquote style="margin-left: 20px; color: blue;">AND resource_type = 'Course'</span> <small><span style="color: grey;">--отфильтровываем только курсы по типу продукта</span></small></blockquote>
<blockquote style="color: blue;">ORDER BY 1, 2;</span> <small><span style="color: grey;"> -- Сортировка результатов по user_id и resource_id</span></small></blockquote><br>  
Результатом запроса будет таблица 'project4.csv'.

#### Далее из полученного файла выясним сколько различных пар курсов встречаются вместе у клиентов в покупках и какая пара курсов самя популярная.

In [1]:
import pandas as pd
import itertools
from itertools import combinations
import numpy as np

In [2]:
df = pd.read_csv(r'project4.csv')
print(df.head(), df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34172 entries, 0 to 34171
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   user_id      34172 non-null  int64
 1   resource_id  34172 non-null  int64
dtypes: int64(2)
memory usage: 534.1 KB
   user_id  resource_id
0       51          516
1       51         1099
2     6117          356
3     6117          357
4     6117         1125 None


#### Рассчет количества разных пар курсов, которые встречаются вместе

In [3]:
# сформируем множество уникальных значений 'resource_id' для каждого пользователя. 
df_pair_count = df.groupby('user_id')['resource_id'].agg( # сгруппируем данные по столбцу 'user_id'
        ['nunique', set]
).sort_values(by='nunique', ascending=False) 

# формируем пары курсов для каждого пользьзователя
courses_pairs= df_pair_count['set'].apply(lambda x: list(itertools.combinations(sorted(x), 2)))
# создадим список пар курсов, для чего объединяем получившиеся пары курсов
courses_pair_list = []
for course in courses_pairs.values:
    for elem in course: 
        courses_pair_list.append(elem) 
        
# считаем количество уникальных пар
result = len(set(courses_pair_list))
display("Количество уникальных пар курсов: {}".format(result))

'Количество уникальных пар курсов: 3989'

#### Выясним какая пара курсов самая популярная

In [4]:
# Создадим DataFrame на основе списков пар курсов
pairs_count=pd.DataFrame(courses_pair_list)
# display(pairs_count.info(), pairs_count.head())
# Преобразуем значения в столбцах 0 и 1 к строковому типу данных
pairs_count[0]=pairs_count[0].astype(str)
pairs_count[1]=pairs_count[1].astype(str)
# Создадим новый столбец 'pair', в котором значения формируются путем объединения строк из столбцов 0 и 1
pairs_count['pair']=pairs_count[[0, 1]].agg(','. join, axis=1)
# Создадим новый DataFrame который содержит уникальные комбинации курсов и их частоту.
result_pairs_count = pairs_count['pair'].value_counts().reset_index()
result_pairs_count.columns = ['pair', 'count']

# Вывод датафрейма
display(result_pairs_count, result_pairs_count.info())
display("Самая популярная пара курсов: {}".format(result_pairs_count.at[0, 'pair']))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3989 entries, 0 to 3988
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   pair    3989 non-null   object
 1   count   3989 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 62.5+ KB


,pair,count
0,"551,566",797
1,"515,551",417
2,"489,551",311
3,"523,551",304
4,"566,794",290
...,...,...
3984,"571,866",1
3985,"368,679",1
3986,"835,1102",1
3987,"809,1125",1


None

'Самая популярная пара курсов: 551,566'

#### Судя по полученной таблице, есть пары курсов которые встречаются редко. Необходимо проверить не являються ли эти данные выбросами, чтобы избежать искажения анализа.<Br>

#### Для проверки данных на выбросы посчитаем квартили: первый квартиль Q1 и третий квартиль Q3.

In [5]:
# Рассчитаем количество покупок различных курсов
courses_count = df['resource_id'].value_counts().reset_index()
courses_count.columns = ['resource_id', 'resource_id_count']
display(courses_count)
# Рассчитаем Q1
Q1 = courses_count['resource_id_count'].quantile(0.25).round()
# Рассчитаем Q3
Q3 = courses_count['resource_id_count'].quantile(0.75).round()
# Рассчитаем интерквартильный размах, который характеризует меру разброса значений внутри выборки
IQR = Q3-Q1
# Поскольку нас интересует проверка только самых редко встречаемых курсов,то рассчитаем только нижнюю границу
low_border = Q1 - 1.5*IQR
display("Q1: {}".format(Q1))
display("Q3: {}".format(Q3))
display("Выбросы:{}".format(low_border))

,resource_id,resource_id_count
0,551,2938
1,566,2345
2,515,1314
3,489,1128
4,490,1088
...,...,...
121,902,3
122,833,3
123,1199,2
124,1201,2


'Q1: 58.0'

'Q3: 350.0'

'Выбросы:-380.0'

#### В данном случае значение выброса получилось отрицательным, следовательно можно предположить, что выбросов в расмматриваемых данных нет.  
#### В данном случае будет логично принять за нижнюю границу значение первого квартиля (Q1), что позволит отбросить данные с низкой частотой и сосредоточиться на наиболее часто встречаемых парах курсов.

### C  учетом исследования данных можно составить рекомендательную таблицу. 


In [14]:
# Преобразуем полученный датафрейм в словарь
result_pairs_dict = result_pairs_count.set_index('pair')
result_pairs_dict = result_pairs_dict['count'].to_dict()
# Вывод словаря
# display(result_pairs_dict)




# Составим список уникальных id курсов, которые есть во всех подобранных парах
lst_id = result_pairs_count['pair'].str.split(',').explode().unique()
# Вывод списка уникальных id курсов
# display(lst_id))
res = {}
for i in lst_id:
    prom = []
    for j, value in result_pairs_dict.items():
        lst_prom = j.split(',')
        if i in lst_prom:
            lst_prom.remove(str(i))
            prom.append((lst_prom[0], value))
    prom_res = []
    max_1 = max(prom, key=lambda x: x[1])
    prom_res.append(max_1[0])
    prom.remove(max_1)
    max_2 = max(prom, key=lambda x: x[1])
    prom_res.append(max_2[0])
    
    res[i] = prom_res


print(res)




# result_rec = {} # Создадим пустой словарь для хранения результатов
# for i in lst_id: # Итерируем по элементам списка lst_id. i будет принимать значения из этого списка
#     temp_pair_dict = [] # Создадим пустой список для временного хранения пар (id купленного курса, частота покупки)
#     for j, value in result_pairs_dict.items(): # Итерируем по парам ключ-значение в словаре 
#         lst_temp_pair_dict = j.split(',') # Разделим строки j по запятой, чтобы создать список из двух значений
#         if i in lst_temp_pair_dict: # Проверим, содержится ли текущее значение i в словаре
#             lst_temp_pair_dict.remove(i)
#             temp_pair_dict.append((lst_temp_pair_dict[0], value)) # Добавим список из двух значений в временный список
#     pair_dict_res = [] # Создадим пустой список для временного хранения результатов текущего i
#     max_1 = max(temp_pair_dict, key=lambda x: x[1]) # Найдем максимальный элемент в списке по второму элементу списка (частота покупок)
#     if max_1[1] > Q1: # Проверим, превышает ли частота покупок максимального id значение Q1
#         pair_dict_res.append(max_1[0]) # Если условие выполняется, добавим первый id в список
#     else: 
#         pair_dict_res.append('551') # Если условие не выполняется, то заменяем текущее значение на id самого покупаемого курса '551'
#     temp_pair_dict.remove(max_1) # Удалим отфильтрованный максимальный id из временного списка
#     max_2 = max(temp_pair_dict, key=lambda x: x[1]) # Найдем новый максимальный id в временном списке
#     if max_2[1] > Q1: # Проверим, превышает ли частота покупок нового максимального id значение Q1
#         pair_dict_res.append(max_2[0]) # Если условие выполняется, добавим второй id в список
#     else: 
#         pair_dict_res.append('566') # Если условие не выполняется, то заменяем текущее значение на id второго по покупаемости курса '566'
    
#         result_rec[i] = pair_dict_res

# Преобразем словарь в DataFrame
recommendations= pd.DataFrame(list(res.items()), columns=['main_course', 'recommendation_1_recommendation_2'])

# Разделение 'recommendation_1_recommendation_2' на две колонки 'recommendation_1' и 'recommendation_2'
recommendations[['recommendation_1', 'recommendation_2']] = pd.DataFrame(recommendations['recommendation_1_recommendation_2'].tolist(), index=recommendations.index)

# Удаление ненужной колонки 'recommendation_1_recommendation_2'
recommendations.drop('recommendation_1_recommendation_2', axis=1, inplace=True)

# Вывод результирующего DataFrame
display(recommendations)


{'551': ['566', '515'], '566': ['551', '794'], '515': ['551', '489'], '489': ['551', '515'], '523': ['551', '515'], '794': ['566', '551'], '490': ['566', '551'], '570': ['752', '507'], '752': ['570', '507'], '569': ['572', '840'], '572': ['569', '504'], '553': ['745', '516'], '745': ['553', '516'], '840': ['569', '572'], '514': ['551', '515'], '516': ['745', '553'], '504': ['572', '569'], '552': ['551', '523'], '507': ['570', '752'], '809': ['490', '570'], '502': ['551', '566'], '571': ['1125', '357'], '1125': ['571', '912'], '357': ['571', '356'], '564': ['523', '551'], '749': ['551', '515'], '777': ['551', '523'], '679': ['551', '489'], '356': ['571', '357'], '568': ['745', '553'], '363': ['511', '562'], '511': ['363', '562'], '764': ['566', '551'], '519': ['551', '523'], '765': ['571', '1125'], '562': ['363', '511'], '1103': ['551', '566'], '912': ['571', '1125'], '1115': ['570', '752'], '1139': ['745', '553'], '743': ['490', '757'], '1161': ['840', '569'], '750': ['551', '566'], '3

,main_course,recommendation_1,recommendation_2
0,551,566,515
1,566,551,794
2,515,551,489
3,489,551,515
4,523,551,515
...,...,...,...
121,911,514,908
122,1201,571,1144
123,902,742,777
124,1182,1187,553


### Вывод по проекту:  

#### В результате исследования данных, выяснили какие пары курсов покупались больше одного раза, определили количество покупок этих пар, выявили самые популярные курсы. В результате полученных данных, выяснили, что некоторые пары курсов продаются намного реже. Самой популярной парой курсов являются курсы с id 551 и 566.
#### Для построения рекомендаций можно предположить, если заменить наименее покупаемые курсы на самые популярные, то это приведет к повышению среднего чека.
#### Для опредения минимальной границы количества покупаемых курсов, посчитали первый квартиль для всех проданных курсов и проверили данные на выбросы. По результатам расчетов в представленных данных выбросов нет, поэтому значение первого квартиля принимаем за минимальную границу количества продаж курсов. 
#### Итоговая рекомендательная таблица построена таким образом, что к основному покупаемому курсу, предлагается еще 2 курса, которые могли бы подойти клиенту. Соответсвенно при ее построении все курсы, продажи которых были ниже значения нижней границы были заменены на курсы из самой поппулярной продаваемой пары. 
